In [1]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Configura el navegador con undetected_chromedriver
browser = uc.Chrome()

# URL inicial: Página de listados de Madrid
url = "https://www.pisos.com/alquiler/pisos-madrid/"
browser.get(url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception as e:
    print("No se encontró el botón de cookies o ya fue aceptado.")

# Espera a que los inmuebles se carguen en la página principal
try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
    )
except Exception as e:
    print("TimeoutException: No se encontraron los inmuebles en el tiempo esperado.")
    browser.quit()

# Extraer el primer enlace de la lista de inmuebles
html = browser.page_source
soup = bs(html, 'lxml')

# Encuentra el primer contenedor de inmueble y obtiene el enlace
primer_inmueble = soup.find('div', {'class': 'ad-preview'})
primer_enlace = primer_inmueble.get('data-lnk-href')
primer_enlace_completo = f"https://www.pisos.com{primer_enlace}"

# Accede al primer enlace de inmueble
browser.get(primer_enlace_completo)
time.sleep(3)  # Espera a que la página del primer inmueble cargue

# Lista para almacenar los datos
all_data = []

# Bucle para iterar desde el primer inmueble a los siguientes inmuebles desde su página
while True:
    try:
        # Extraer la información del inmueble actual
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')

        # Extraer datos del inmueble
        try:
            # Precio
            precio = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}).text.split(' ')[0] if soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}) else "N/A"

            # Superficie construida
            superficie_construida = soup_inmueble.find('span', {'class': 'features__value'}).text.split(' ')[0] if soup_inmueble.find('span', {'class': 'features__value'}) else "N/A"

            # Última actualización
            ultima_actualizacion = soup_inmueble.find('p', {'class': 'last-update__date'}).text if soup_inmueble.find('p', {'class': 'last-update__date'}) else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'}).get_text(strip=True)
                    value = feature.find('span', {'class': 'features__value'}).get_text(strip=True)
                    features_list.append((label, value))

            # Certificado energético
            try:
                energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
                consumo = energy_certificate.find('div', {'class': 'energy-certificate__data'}).find_all('span')[1].get_text(strip=True)
                emisiones = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})[1].find_all('span')[1].get_text(strip=True)
            except AttributeError:
                consumo = "N/A"
                emisiones = "N/A"

            # Guarda los datos en una lista
            data = {
                'Enlace': browser.current_url,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'Alquiler'  # Se añade la operación como compra
            }

            all_data.append(data)

            # Imprime los datos de la página actual (opcional)
            print(f"Inmueble {len(all_data)}: {data}")

        except Exception as e:
            print(f"Error al extraer datos de {browser.current_url}: {e}")

        # Intenta hacer clic en el botón de "Siguiente" para ir al siguiente inmueble
        try:
            siguiente_boton = WebDriverWait(browser, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".navigation__link.navigation__link--next"))
            )
            siguiente_boton.click()

            # Pausa para esperar a que cargue la siguiente página
            time.sleep(random.uniform(3, 5))

        except Exception as e:
            # Si no se encuentra el botón "Siguiente", termina el bucle
            print("No se encontró el botón 'Siguiente' o ya no hay más inmuebles.")
            break

    except Exception as e:
        print(f"Error al cargar la página del inmueble: {e}")
        break

# Una vez terminado el bucle, puedes convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('pisos_alquiler_30_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()

Error al extraer datos de https://www.pisos.com/alquilar/piso-la_finca-47557750571_280500/: 'NoneType' object has no attribute 'get_text'
No se encontró el botón 'Siguiente' o ya no hay más inmuebles.
Empty DataFrame
Columns: []
Index: []


In [3]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Configura el navegador con undetected_chromedriver
browser = uc.Chrome()

# URL inicial: Página de listados de alquiler en Madrid
url = "https://www.pisos.com/alquiler/pisos-madrid/"
browser.get(url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception as e:
    print("No se encontró el botón de cookies o ya fue aceptado.")

# Espera a que los inmuebles se carguen en la página principal
try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
    )
except Exception as e:
    print("TimeoutException: No se encontraron los inmuebles en el tiempo esperado.")
    browser.quit()

# Extraer el primer enlace de la lista de inmuebles
html = browser.page_source
soup = bs(html, 'lxml')

# Encuentra el primer contenedor de inmueble y obtiene el enlace
primer_inmueble = soup.find('div', {'class': 'ad-preview'})
primer_enlace = primer_inmueble.get('data-lnk-href')
primer_enlace_completo = f"https://www.pisos.com{primer_enlace}"

# Accede al primer enlace de inmueble
browser.get(primer_enlace_completo)
time.sleep(3)  # Espera a que la página del primer inmueble cargue

# Lista para almacenar los datos
all_data = []

# Contador de inmuebles visitados
contador_inmuebles = 0
max_inmuebles = 30  # Limitar el scraping a 30 inmuebles para el muestreo

# Bucle para iterar desde el primer inmueble a los siguientes inmuebles desde su página
while contador_inmuebles < max_inmuebles:
    try:
        # Extraer la información del inmueble actual
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')

        # Extraer datos del inmueble
        try:
            # Precio
            precio = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}).text.split(' ')[0] if soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}) else "N/A"

            # Superficie construida
            superficie_construida = soup_inmueble.find('span', {'class': 'features__value'}).text.split(' ')[0] if soup_inmueble.find('span', {'class': 'features__value'}) else "N/A"

            # Última actualización
            ultima_actualizacion = soup_inmueble.find('p', {'class': 'last-update__date'}).text if soup_inmueble.find('p', {'class': 'last-update__date'}) else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'}).get_text(strip=True)
                    value = feature.find('span', {'class': 'features__value'}).get_text(strip=True)
                    features_list.append((label, value))

            # Certificado energético
            try:
                energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
                consumo = energy_certificate.find('div', {'class': 'energy-certificate__data'}).find_all('span')[1].get_text(strip=True)
                emisiones = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})[1].find_all('span')[1].get_text(strip=True)
            except AttributeError:
                consumo = "N/A"
                emisiones = "N/A"

            # Guarda los datos en una lista
            data = {
                'Enlace': browser.current_url,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'Alquiler'
            }

            all_data.append(data)
            contador_inmuebles += 1  # Incrementa el contador de inmuebles procesados

            # Imprime los datos de la página actual (opcional)
            print(f"Inmueble {contador_inmuebles}: {data}")

        except Exception as e:
            print(f"Error al extraer datos de {browser.current_url}: {e}")

        # Si ya hemos procesado 30 inmuebles, romper el bucle
        if contador_inmuebles >= max_inmuebles:
            break

        # Scroll hacia abajo para asegurarse de que el botón de "Siguiente" esté visible
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Intenta hacer clic en el botón de "Siguiente" para ir al siguiente inmueble
        try:
            siguiente_boton = WebDriverWait(browser, 30).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".navigation__link.navigation__link--next"))
            )
            # Clic usando JavaScript
            browser.execute_script("arguments[0].click();", siguiente_boton)

            # Pausa para esperar a que cargue la siguiente página
            time.sleep(random.uniform(3, 5))

        except Exception as e:
            # Si no se encuentra el botón "Siguiente", termina el bucle
            print("No se encontró el botón 'Siguiente' o ya no hay más inmuebles.")
            break

    except Exception as e:
        print(f"Error al cargar la página del inmueble: {e}")
        break

# Una vez terminado el bucle, puedes convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('pisos_alquiler_30_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()


Error al extraer datos de https://www.pisos.com/alquilar/piso-la_finca-47557750571_280500/: 'NoneType' object has no attribute 'get_text'
No se encontró el botón 'Siguiente' o ya no hay más inmuebles.
Empty DataFrame
Columns: []
Index: []


In [4]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Configura el navegador con undetected_chromedriver
browser = uc.Chrome()

# URL inicial: Página de listados de alquiler en Madrid
url = "https://www.pisos.com/alquiler/pisos-madrid/"
browser.get(url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception as e:
    print("No se encontró el botón de cookies o ya fue aceptado.")

# Espera a que los inmuebles se carguen en la página principal
try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
    )
except Exception as e:
    print("TimeoutException: No se encontraron los inmuebles en el tiempo esperado.")
    browser.quit()

# Extraer el primer enlace de la lista de inmuebles
html = browser.page_source
soup = bs(html, 'lxml')

# Encuentra el primer contenedor de inmueble y obtiene el enlace
primer_inmueble = soup.find('div', {'class': 'ad-preview'})
primer_enlace = primer_inmueble.get('data-lnk-href')
primer_enlace_completo = f"https://www.pisos.com{primer_enlace}"

# Accede al primer enlace de inmueble
browser.get(primer_enlace_completo)
time.sleep(3)  # Espera a que la página del primer inmueble cargue

# Lista para almacenar los datos
all_data = []

# Contador de inmuebles visitados
contador_inmuebles = 0
max_inmuebles = 30  # Limitar el scraping a 30 inmuebles para el muestreo

# Bucle para iterar desde el primer inmueble a los siguientes inmuebles desde su página
while contador_inmuebles < max_inmuebles:
    try:
        # Extraer la información del inmueble actual
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')

        # Extraer datos del inmueble
        try:
            # Comprobamos si el precio existe antes de extraerlo
            precio_elemento = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'})
            precio = precio_elemento.text.split(' ')[0] if precio_elemento else "N/A"

            # Comprobamos si la superficie construida existe
            superficie_elemento = soup_inmueble.find('span', {'class': 'features__value'})
            superficie_construida = superficie_elemento.text.split(' ')[0] if superficie_elemento else "N/A"

            # Comprobamos si la última actualización existe
            actualizacion_elemento = soup_inmueble.find('p', {'class': 'last-update__date'})
            ultima_actualizacion = actualizacion_elemento.text if actualizacion_elemento else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label_element = feature.find('span', {'class': 'features__label'})
                    value_element = feature.find('span', {'class': 'features__value'})
                    label = label_element.get_text(strip=True) if label_element else "N/A"
                    value = value_element.get_text(strip=True) if value_element else "N/A"
                    features_list.append((label, value))

            # Certificado energético (comprobamos si existe antes de extraer)
            try:
                energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
                consumo = energy_certificate.find('div', {'class': 'energy-certificate__data'}).find_all('span')[1].get_text(strip=True) if energy_certificate else "N/A"
                emisiones = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})[1].find_all('span')[1].get_text(strip=True) if energy_certificate else "N/A"
            except (AttributeError, IndexError):
                consumo = "N/A"
                emisiones = "N/A"

            # Guarda los datos en una lista
            data = {
                'Enlace': browser.current_url,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'Alquiler'
            }

            all_data.append(data)
            contador_inmuebles += 1  # Incrementa el contador de inmuebles procesados

            # Imprime los datos de la página actual (opcional)
            print(f"Inmueble {contador_inmuebles}: {data}")

        except Exception as e:
            print(f"Error al extraer datos de {browser.current_url}: {e}")

        # Si ya hemos procesado 30 inmuebles, romper el bucle
        if contador_inmuebles >= max_inmuebles:
            break

        # Scroll hacia abajo para asegurarse de que el botón de "Siguiente" esté visible
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Intenta hacer clic en el botón de "Siguiente" para ir al siguiente inmueble
        try:
            siguiente_boton = WebDriverWait(browser, 30).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".navigation__link.navigation__link--next"))
            )
            # Clic usando JavaScript
            browser.execute_script("arguments[0].click();", siguiente_boton)

            # Pausa para esperar a que cargue la siguiente página
            time.sleep(random.uniform(3, 5))

        except Exception as e:
            # Si no se encuentra el botón "Siguiente", termina el bucle
            print("No se encontró el botón 'Siguiente' o ya no hay más inmuebles.")
            break

    except Exception as e:
        print(f"Error al cargar la página del inmueble: {e}")
        break

# Una vez terminado el bucle, puedes convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('pisos_alquiler_30_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()


Inmueble 1: {'Enlace': 'https://www.pisos.com/alquilar/piso-la_finca-47557750571_280500/', 'Precio': '5.500', 'Superficie Construida': '190', 'Última Actualización': 'Anuncio actualizado el 24/10/2024', 'Consumo Energético': 'Consumo:15 kWh/m² año', 'Emisiones CO2': 'Emisiones:2 Kg CO₂/m² año', 'Características': [('Superficie construida:', '190 m²'), ('Superficie útil:', '170 m²'), ('Habitaciones:', '3'), ('Baños:', '3'), ('Planta:', '2ª'), ('Exterior', 'N/A'), ('Antigüedad:', 'Menos de 5 años'), ('Conservación:', 'En buen estado'), ('Gastos de comunidad:', 'Incluidos'), ('Referencia:', '2805-002404')], 'Tipo de operación': 'Alquiler'}
No se encontró el botón 'Siguiente' o ya no hay más inmuebles.
                                              Enlace Precio  \
0  https://www.pisos.com/alquilar/piso-la_finca-4...  5.500   

  Superficie Construida               Última Actualización  \
0                   190  Anuncio actualizado el 24/10/2024   

      Consumo Energético              Em

In [5]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Función para iniciar el navegador
def iniciar_navegador():
    return uc.Chrome()

# Función para cargar una página y manejar posibles errores
def cargar_pagina(browser, url):
    try:
        browser.get(url)
        return True
    except Exception as e:
        print(f"Error al cargar la página: {e}")
        return False

# Iniciar el navegador
browser = iniciar_navegador()

# URL inicial: Página de listados de alquiler en Madrid
url = "https://www.pisos.com/alquiler/pisos-madrid/"
if not cargar_pagina(browser, url):
    browser = iniciar_navegador()
    cargar_pagina(browser, url)

# Manejar las cookies si aparecen
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception as e:
    print("No se encontró el botón de cookies o ya fue aceptado.")

# Lista para almacenar los datos
all_data = []

# Contador de inmuebles visitados
contador_inmuebles = 0
max_inmuebles = 30  # Limitar el scraping a 30 inmuebles para el muestreo

# Bucle para iterar sobre los inmuebles en la página de alquiler
while contador_inmuebles < max_inmuebles:
    try:
        # Espera a que los inmuebles se carguen en la página
        WebDriverWait(browser, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
        )

        # Extraer el HTML de la página
        html = browser.page_source
        soup = bs(html, 'lxml')

        # Encuentra todos los inmuebles de la página
        inmuebles = soup.find_all('div', {'class': 'ad-preview'})

        # Iterar sobre los inmuebles de la página
        for inmueble in inmuebles:
            if contador_inmuebles >= max_inmuebles:
                break  # Detiene el bucle si ya se ha alcanzado el límite

            enlace_inmueble = inmueble.get('data-lnk-href')
            enlace_completo = f"https://www.pisos.com{enlace_inmueble}"

            # Accede al enlace del inmueble
            if not cargar_pagina(browser, enlace_completo):
                browser = iniciar_navegador()
                cargar_pagina(browser, enlace_completo)

            time.sleep(3)  # Espera a que la página del inmueble cargue

            # Extraer la información del inmueble
            try:
                html_inmueble = browser.page_source
                soup_inmueble = bs(html_inmueble, 'lxml')

                # Comprobamos si el precio existe antes de extraerlo
                precio_elemento = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'})
                precio = precio_elemento.text.split(' ')[0] if precio_elemento else "N/A"

                # Comprobamos si la superficie construida existe
                superficie_elemento = soup_inmueble.find('span', {'class': 'features__value'})
                superficie_construida = superficie_elemento.text.split(' ')[0] if superficie_elemento else "N/A"

                # Comprobamos si la última actualización existe
                actualizacion_elemento = soup_inmueble.find('p', {'class': 'last-update__date'})
                ultima_actualizacion = actualizacion_elemento.text if actualizacion_elemento else "N/A"

                # Características
                c1 = soup_inmueble.find('div', {'class': 'features__content'})
                features_list = []
                if c1:
                    features = c1.find_all('div', {'class': 'features__feature'})
                    for feature in features:
                        label_element = feature.find('span', {'class': 'features__label'})
                        value_element = feature.find('span', {'class': 'features__value'})
                        label = label_element.get_text(strip=True) if label_element else "N/A"
                        value = value_element.get_text(strip=True) if value_element else "N/A"
                        features_list.append((label, value))

                # Certificado energético
                try:
                    energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
                    consumo = energy_certificate.find('div', {'class': 'energy-certificate__data'}).find_all('span')[1].get_text(strip=True) if energy_certificate else "N/A"
                    emisiones = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})[1].find_all('span')[1].get_text(strip=True) if energy_certificate else "N/A"
                except (AttributeError, IndexError):
                    consumo = "N/A"
                    emisiones = "N/A"

                # Guarda los datos en una lista
                data = {
                    'Enlace': browser.current_url,
                    'Precio': precio,
                    'Superficie Construida': superficie_construida,
                    'Última Actualización': ultima_actualizacion,
                    'Consumo Energético': consumo,
                    'Emisiones CO2': emisiones,
                    'Características': features_list,
                    'Tipo de operación': 'Alquiler'
                }

                all_data.append(data)
                contador_inmuebles += 1  # Incrementa el contador de inmuebles procesados

                # Imprime los datos de la página actual (opcional)
                print(f"Inmueble {contador_inmuebles}: {data}")

            except Exception as e:
                print(f"Error al extraer datos de {browser.current_url}: {e}")

        # Intentar hacer clic en el botón de "Siguiente" para pasar a la siguiente página de listados
        try:
            siguiente_boton = WebDriverWait(browser, 30).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".navigation__link.navigation__link--next"))
            )
            siguiente_boton.click()

            # Pausa breve para evitar sobrecargar el servidor
            time.sleep(random.uniform(3, 5))

        except Exception as e:
            # Si no se encuentra el botón "Siguiente", sal del bucle
            print("No se encontró el botón 'Siguiente' o ya no hay más inmuebles.")
            break

    except Exception as e:
        print(f"Error al cargar la página de listados: {e}")
        break

# Convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Guardar el DataFrame en un archivo CSV
df.to_csv('pisos_alquiler_30_inmuebles.csv', index=False)

# Cerrar el navegador
browser.quit()


Inmueble 1: {'Enlace': 'https://www.pisos.com/alquilar/piso-la_finca-47557750571_280500/', 'Precio': '5.500', 'Superficie Construida': '190', 'Última Actualización': 'Anuncio actualizado el 24/10/2024', 'Consumo Energético': 'Consumo:15 kWh/m² año', 'Emisiones CO2': 'Emisiones:2 Kg CO₂/m² año', 'Características': [('Superficie construida:', '190 m²'), ('Superficie útil:', '170 m²'), ('Habitaciones:', '3'), ('Baños:', '3'), ('Planta:', '2ª'), ('Exterior', 'N/A'), ('Antigüedad:', 'Menos de 5 años'), ('Conservación:', 'En buen estado'), ('Gastos de comunidad:', 'Incluidos'), ('Referencia:', '2805-002404')], 'Tipo de operación': 'Alquiler'}
Inmueble 2: {'Enlace': 'https://www.pisos.com/alquilar/casa_unifamiliar-las_matas_los_penascales-29189878080_280500/', 'Precio': '4.000', 'Superficie Construida': '600', 'Última Actualización': 'Anuncio actualizado el 24/10/2024', 'Consumo Energético': 'N/A', 'Emisiones CO2': 'N/A', 'Características': [('Superficie construida:', '600 m²'), ('Superficie 

In [6]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Función para iniciar el navegador
def iniciar_navegador():
    return uc.Chrome()

# Función para cargar una página y manejar posibles errores
def cargar_pagina(browser, url):
    try:
        browser.get(url)
        return True
    except Exception as e:
        print(f"Error al cargar la página: {e}")
        return False

# Iniciar el navegador
browser = iniciar_navegador()

# URL inicial: Página de listados de alquiler en Madrid
url = "https://www.pisos.com/alquiler/pisos-madrid/"
if not cargar_pagina(browser, url):
    browser = iniciar_navegador()
    cargar_pagina(browser, url)

# Manejar las cookies si aparecen
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception as e:
    print("No se encontró el botón de cookies o ya fue aceptado.")

# Lista para almacenar los datos
all_data = []

# Contador de inmuebles visitados
contador_inmuebles = 0
max_inmuebles = 200  # Cambiado a 200 inmuebles

# Bucle para iterar sobre los inmuebles en la página de alquiler
while contador_inmuebles < max_inmuebles:
    try:
        # Espera a que los inmuebles se carguen en la página
        WebDriverWait(browser, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
        )

        # Extraer el HTML de la página
        html = browser.page_source
        soup = bs(html, 'lxml')

        # Encuentra todos los inmuebles de la página
        inmuebles = soup.find_all('div', {'class': 'ad-preview'})

        # Iterar sobre los inmuebles de la página
        for inmueble in inmuebles:
            if contador_inmuebles >= max_inmuebles:
                break  # Detiene el bucle si ya se ha alcanzado el límite

            enlace_inmueble = inmueble.get('data-lnk-href')
            enlace_completo = f"https://www.pisos.com{enlace_inmueble}"

            # Accede al enlace del inmueble
            if not cargar_pagina(browser, enlace_completo):
                browser = iniciar_navegador()
                cargar_pagina(browser, enlace_completo)

            time.sleep(3)  # Espera a que la página del inmueble cargue

            # Extraer la información del inmueble
            try:
                html_inmueble = browser.page_source
                soup_inmueble = bs(html_inmueble, 'lxml')

                # Comprobamos si el precio existe antes de extraerlo
                precio_elemento = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'})
                precio = precio_elemento.text.split(' ')[0] if precio_elemento else "N/A"

                # Comprobamos si la superficie construida existe
                superficie_elemento = soup_inmueble.find('span', {'class': 'features__value'})
                superficie_construida = superficie_elemento.text.split(' ')[0] if superficie_elemento else "N/A"

                # Comprobamos si la última actualización existe
                actualizacion_elemento = soup_inmueble.find('p', {'class': 'last-update__date'})
                ultima_actualizacion = actualizacion_elemento.text if actualizacion_elemento else "N/A"

                # Características
                c1 = soup_inmueble.find('div', {'class': 'features__content'})
                features_list = []
                if c1:
                    features = c1.find_all('div', {'class': 'features__feature'})
                    for feature in features:
                        label_element = feature.find('span', {'class': 'features__label'})
                        value_element = feature.find('span', {'class': 'features__value'})
                        label = label_element.get_text(strip=True) if label_element else "N/A"
                        value = value_element.get_text(strip=True) if value_element else "N/A"
                        features_list.append((label, value))

                # Certificado energético
                try:
                    energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
                    consumo = energy_certificate.find('div', {'class': 'energy-certificate__data'}).find_all('span')[1].get_text(strip=True) if energy_certificate else "N/A"
                    emisiones = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})[1].find_all('span')[1].get_text(strip=True) if energy_certificate else "N/A"
                except (AttributeError, IndexError):
                    consumo = "N/A"
                    emisiones = "N/A"

                # Guarda los datos en una lista
                data = {
                    'Enlace': browser.current_url,
                    'Precio': precio,
                    'Superficie Construida': superficie_construida,
                    'Última Actualización': ultima_actualizacion,
                    'Consumo Energético': consumo,
                    'Emisiones CO2': emisiones,
                    'Características': features_list,
                    'Tipo de operación': 'Alquiler'
                }

                all_data.append(data)
                contador_inmuebles += 1  # Incrementa el contador de inmuebles procesados

                # Imprime los datos de la página actual (opcional)
                print(f"Inmueble {contador_inmuebles}: {data}")

            except Exception as e:
                print(f"Error al extraer datos de {browser.current_url}: {e}")

        # Intentar hacer clic en el botón de "Siguiente" para pasar a la siguiente página de listados
        try:
            siguiente_boton = WebDriverWait(browser, 30).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".navigation__link.navigation__link--next"))
            )
            siguiente_boton.click()

            # Pausa breve para evitar sobrecargar el servidor
            time.sleep(random.uniform(3, 7))  # Tiempo de espera aleatorio para evitar bloqueos

        except Exception as e:
            # Si no se encuentra el botón "Siguiente", sal del bucle
            print("No se encontró el botón 'Siguiente' o ya no hay más inmuebles.")
            break

    except Exception as e:
        print(f"Error al cargar la página de listados: {e}")
        break

# Convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Guardar el DataFrame en un archivo CSV
df.to_csv('pisos_alquiler_200_inmuebles.csv', index=False)

# Cerrar el navegador
browser.quit()


Inmueble 1: {'Enlace': 'https://www.pisos.com/alquilar/piso-la_finca-47557750571_280500/', 'Precio': '5.500', 'Superficie Construida': '190', 'Última Actualización': 'Anuncio actualizado el 24/10/2024', 'Consumo Energético': 'Consumo:15 kWh/m² año', 'Emisiones CO2': 'Emisiones:2 Kg CO₂/m² año', 'Características': [('Superficie construida:', '190 m²'), ('Superficie útil:', '170 m²'), ('Habitaciones:', '3'), ('Baños:', '3'), ('Planta:', '2ª'), ('Exterior', 'N/A'), ('Antigüedad:', 'Menos de 5 años'), ('Conservación:', 'En buen estado'), ('Gastos de comunidad:', 'Incluidos'), ('Referencia:', '2805-002404')], 'Tipo de operación': 'Alquiler'}
Inmueble 2: {'Enlace': 'https://www.pisos.com/alquilar/piso-zona_avenida_de_europa28224-45068829832_101800/', 'Precio': '1.400', 'Superficie Construida': '65', 'Última Actualización': 'Anuncio actualizado el 24/10/2024', 'Consumo Energético': 'N/A', 'Emisiones CO2': 'N/A', 'Características': [('Superficie construida:', '65 m²'), ('Habitaciones:', '1'), 

In [7]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Función para iniciar el navegador
def iniciar_navegador():
    return uc.Chrome()

# Función para cargar una página y manejar posibles errores
def cargar_pagina(browser, url):
    try:
        browser.get(url)
        return True
    except Exception as e:
        print(f"Error al cargar la página: {e}")
        return False

# Iniciar el navegador
browser = iniciar_navegador()

# URL inicial: Página de listados de alquiler en Madrid
url = "https://www.pisos.com/alquiler/pisos-madrid/"
if not cargar_pagina(browser, url):
    browser = iniciar_navegador()
    cargar_pagina(browser, url)

# Manejar las cookies si aparecen
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception as e:
    print("No se encontró el botón de cookies o ya fue aceptado.")

# Lista para almacenar los datos
all_data = []

# Contador de inmuebles visitados
contador_inmuebles = 0
max_inmuebles = 200  # Cambiado a 200 inmuebles

# Bucle para iterar sobre los inmuebles en la página de alquiler
while contador_inmuebles < max_inmuebles:
    try:
        # Espera a que los inmuebles se carguen en la página
        WebDriverWait(browser, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
        )

        # Extraer el HTML de la página
        html = browser.page_source
        soup = bs(html, 'lxml')

        # Encuentra todos los inmuebles de la página
        inmuebles = soup.find_all('div', {'class': 'ad-preview'})

        # Iterar sobre los inmuebles de la página
        for inmueble in inmuebles:
            if contador_inmuebles >= max_inmuebles:
                break  # Detiene el bucle si ya se ha alcanzado el límite

            enlace_inmueble = inmueble.get('data-lnk-href')
            enlace_completo = f"https://www.pisos.com{enlace_inmueble}"

            # Accede al enlace del inmueble
            if not cargar_pagina(browser, enlace_completo):
                browser = iniciar_navegador()
                cargar_pagina(browser, enlace_completo)

            time.sleep(3)  # Espera a que la página del inmueble cargue

            # Extraer la información del inmueble
            try:
                html_inmueble = browser.page_source
                soup_inmueble = bs(html_inmueble, 'lxml')

                # Comprobamos si el precio existe antes de extraerlo
                precio_elemento = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'})
                precio = precio_elemento.text.split(' ')[0] if precio_elemento else "N/A"

                # Comprobamos si la superficie construida existe
                superficie_elemento = soup_inmueble.find('span', {'class': 'features__value'})
                superficie_construida = superficie_elemento.text.split(' ')[0] if superficie_elemento else "N/A"

                # Comprobamos si la última actualización existe
                actualizacion_elemento = soup_inmueble.find('p', {'class': 'last-update__date'})
                ultima_actualizacion = actualizacion_elemento.text if actualizacion_elemento else "N/A"

                # Características
                c1 = soup_inmueble.find('div', {'class': 'features__content'})
                features_list = []
                if c1:
                    features = c1.find_all('div', {'class': 'features__feature'})
                    for feature in features:
                        label_element = feature.find('span', {'class': 'features__label'})
                        value_element = feature.find('span', {'class': 'features__value'})
                        label = label_element.get_text(strip=True) if label_element else "N/A"
                        value = value_element.get_text(strip=True) if value_element else "N/A"
                        features_list.append((label, value))

                # Certificado energético
                try:
                    energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
                    consumo = energy_certificate.find('div', {'class': 'energy-certificate__data'}).find_all('span')[1].get_text(strip=True) if energy_certificate else "N/A"
                    emisiones = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})[1].find_all('span')[1].get_text(strip=True) if energy_certificate else "N/A"
                except (AttributeError, IndexError):
                    consumo = "N/A"
                    emisiones = "N/A"

                # Guarda los datos en una lista
                data = {
                    'Enlace': browser.current_url,
                    'Precio': precio,
                    'Superficie Construida': superficie_construida,
                    'Última Actualización': ultima_actualizacion,
                    'Consumo Energético': consumo,
                    'Emisiones CO2': emisiones,
                    'Características': features_list,
                    'Tipo de operación': 'Alquiler'
                }

                all_data.append(data)
                contador_inmuebles += 1  # Incrementa el contador de inmuebles procesados

                # Imprime los datos de la página actual (opcional)
                print(f"Inmueble {contador_inmuebles}: {data}")

            except Exception as e:
                print(f"Error al extraer datos de {browser.current_url}: {e}")

        # Intentar hacer clic en el botón de "Siguiente" para pasar a la siguiente página de listados
        try:
            siguiente_boton = WebDriverWait(browser, 30).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".navigation__link.navigation__link--next"))
            )
            siguiente_boton.click()
            print("Pasando a la siguiente página...")  # Agregar mensaje de depuración

            # Pausa breve para evitar sobrecargar el servidor
            time.sleep(random.uniform(3, 7))  # Tiempo de espera aleatorio para evitar bloqueos

        except Exception as e:
            # Si no se encuentra el botón "Siguiente", sal del bucle
            print("No se encontró el botón 'Siguiente' o ya no hay más inmuebles.")
            break

    except Exception as e:
        print(f"Error al cargar la página de listados: {e}")
        break

# Convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Guardar el DataFrame en un archivo CSV
df.to_csv('pisos_alquiler_200_inmuebles.csv', index=False)

# Cerrar el navegador
browser.quit()


Inmueble 1: {'Enlace': 'https://www.pisos.com/alquilar/piso-la_finca-47557750571_280500/', 'Precio': '5.500', 'Superficie Construida': '190', 'Última Actualización': 'Anuncio actualizado el 24/10/2024', 'Consumo Energético': 'Consumo:15 kWh/m² año', 'Emisiones CO2': 'Emisiones:2 Kg CO₂/m² año', 'Características': [('Superficie construida:', '190 m²'), ('Superficie útil:', '170 m²'), ('Habitaciones:', '3'), ('Baños:', '3'), ('Planta:', '2ª'), ('Exterior', 'N/A'), ('Antigüedad:', 'Menos de 5 años'), ('Conservación:', 'En buen estado'), ('Gastos de comunidad:', 'Incluidos'), ('Referencia:', '2805-002404')], 'Tipo de operación': 'Alquiler'}
Inmueble 2: {'Enlace': 'https://www.pisos.com/alquilar/piso-zona_avenida_de_europa28224-45068829832_101800/', 'Precio': '1.400', 'Superficie Construida': '65', 'Última Actualización': 'Anuncio actualizado el 24/10/2024', 'Consumo Energético': 'N/A', 'Emisiones CO2': 'N/A', 'Características': [('Superficie construida:', '65 m²'), ('Habitaciones:', '1'), 